In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from torch import cuda, bfloat16
import transformers
import os
from torch import cuda, bfloat16
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
# bnb_config = transformers.BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type='nf4',
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=bfloat16
# )

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# Need auth token for these
hf_auth = os.environ.get('hf_token')
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    # quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

/home/hb/myenv/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Model loaded on cuda:0


In [5]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

/home/hb/myenv/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## Table data

In [8]:
table_data_format = {
    "instruction": "The goal for this task is to identify the type of BGP anomaly based on the given data. The context, section, and table columns provide important information for identifying the correct anomaly type.",
    "input_seg": "[TLE] The context is about BGP data analysis for detecting anomalies. The section is related to a specific time period of BGP monitoring. [TAB] col: | timestamp | asn | num_routes | num_new_routes | num_withdrawals | num_origin_changes | num_route_changes | max_path_length | avg_path_length | max_edit_distance | avg_edit_distance | num_announcements | num_unique_prefixes_announced | row 1: | 2024-06-10 12:00:00 | 12345 | 100 | 5 | 3 | 1 | 2 | 4 | 3.5 | 2 | 1.5 | 10 | 8 | [SEP] row 2: | 2024-06-10 12:05:00 | 12346 | 95 | 7 | 4 | 0 | 1 | 5 | 3.2 | 1 | 1.7 | 9 | 7 | [SEP] row 3: | 2024-06-10 12:10:00 | 12347 | 102 | 4 | 5 | 2 | 3 | 4.1 | 3 | 2.5 | 11 | 10 |",
    "question": "Based on the data provided, identify the type of BGP anomaly. The possible anomaly types are: route_flapping, outage, prefix_hijacking, normal. The data contains various metrics such as num_routes, num_new_routes, num_withdrawals, etc.",
    "output": "route_flapping"
}

In [10]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = f"""
{table_data_format["instruction"]}. 
{table_data_format["input_seg"]}
{table_data_format["question"]}
"""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1024)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 
The goal for this task is to identify the type of BGP anomaly based on the given data. The context, section, and table columns provide important information for identifying the correct anomaly type.. 
[TLE] The context is about BGP data analysis for detecting anomalies. The section is related to a specific time period of BGP monitoring. [TAB] col: | timestamp | asn | num_routes | num_new_routes | num_withdrawals | num_origin_changes | num_route_changes | max_path_length | avg_path_length | max_edit_distance | avg_edit_distance | num_announcements | num_unique_prefixes_announced | row 1: | 2024-06-10 12:00:00 | 12345 | 100 | 5 | 3 | 1 | 2 | 4 | 3.5 | 2 | 1.5 | 10 | 8 | [SEP] row 2: | 2024-06-10 12:05:00 | 12346 | 95 | 7 | 4 | 0 | 1 | 5 | 3.2 | 1 | 1.7 | 9 | 7 | [SEP] row 3: | 2024-06-10 12:10:00 | 12347 | 102 | 4 | 5 | 2 | 3 | 4.1 | 3 | 2.5 | 11 | 10 |
Based on the data provided, identify the type of BGP anomaly. The possible anomaly types are: route_flapping, outage, prefix_

## DDoS

In [ ]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Create a Python script with the Scapy library to detect DDoS attacks from a PCAP file. The script should read the file, analyze traffic patterns by monitoring packets per second against a set threshold within defined time windows, and reset counters for each time window. It should also log any potential attacks showing the source IP and packet rate. Include a final check for the last time window at the end of the file."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=712)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Generate Python code to calculate and visualize 5G network performance KPIs: Total Network Capacity, Capacity per Area, Capacity per Point, Cost per Capacity, Cost per Area, and Surplus per Area. Load data from '5G_Infrastructure/demand_driven_postcode_data_results.csv'. Use keywords: 'capacity', 'cost', 'area', 'numpoints' to identify relevant columns."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=712)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Generate Python code to calculate and visualize 5G network performance KPIs: Total Network Capacity, Capacity per Area, Capacity per Point, Cost per Capacity, Cost per Area, and Surplus per Area. Load data from '5G_Infrastructure/demand_driven_postcode_data_results.csv'. Use keywords: 'capacity', 'cost', 'area', 'numpoints' to identify relevant columns. [/INST]  Sure! Here is some Python code that can be used to calculate and visualize 5G network performance KPIs based on the provided CSV file:
```
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data from the CSV file
df = pd.read_csv('5G_Infrastructure/demand_driven_postcode_data_results.csv')

# Identify the relevant columns
capacity = df['capacity']
cost = df['cost']
area = df['area']
numpoints = df['numpoints']

# Calculate the KPIs
total_network_capacity = capacity.sum()
capacity_per_area = capacity / area
capacity_per_point = capacity / numpoints
cost_per_capacity = cost / capacity


## Multiple Choice
`5G configuration analysis v1`:
Accuracy: 66%
Correct answers: 13 + 7 = 20
Incorrect answers: 10

`5G configuration analysis v2`:
Accuracy: 56%
Correct answers: 17
Incorrect answers: 13